In [43]:
from __future__ import print_function
import time

import math
import numpy as np

import time 
from pyspark.mllib.linalg.distributed import *
import numpy as np
from numpy.random import randint
from numpy.linalg import norm, eigh
from numpy.linalg import svd as SVD
from numpy.fft import fft, ifft
from pyspark.sql import SparkSession
from pyspark.mllib.linalg import DenseMatrix, DenseVector
from pyspark.mllib.linalg.distributed import RowMatrix,IndexedRowMatrix



from pyspark.mllib.linalg import DenseMatrix, DenseVector


def zscore(a, axis=0, ddof=0):
    # the same as python implementation
    a = np.asanyarray(a)
    mns = a.mean(axis=axis)
    sstd = a.std(axis=axis, ddof=ddof)
    if axis and mns.ndim < a.ndim:
        res = ((a - np.expand_dims(mns, axis=axis)) /
               np.expand_dims(sstd, axis=axis))
    else:
        res = (a - mns) / sstd
    return np.nan_to_num(res)


def roll_zeropad(a, shift, axis=None):
    # the same as python implementation
    a = np.asanyarray(a)
    if shift == 0:
        return a
    if axis is None:
        n = a.size
        reshape = True
    else:
        n = a.shape[axis]
        reshape = False
    if np.abs(shift) > n:
        res = np.zeros_like(a)
    elif shift < 0:
        shift += n
        zeros = np.zeros_like(a.take(np.arange(n-shift), axis))
        res = np.concatenate((a.take(np.arange(n-shift, n), axis), zeros), axis)
    else:
        zeros = np.zeros_like(a.take(np.arange(n-shift, n), axis))
        res = np.concatenate((zeros, a.take(np.arange(n-shift), axis)), axis)
    if reshape:
        return res.reshape(a.shape)
    else:
        return res
        for i in range(len(idx)):
            dist = float('inf')
            c = x[i]

            for l,j in enumerate(centroids):
                d,_ = _sbd(x[i],j)
                if dist > d:
                    c = j
            idx[i] = l



def _ncc_c(x, y):
    # the same as python implementation    
    den = np.array(norm(x) * norm(y))
    den[den == 0] = np.Inf
    x_len = len(x)
    fft_size = 1 << (2*x_len-1).bit_length()
    cc = ifft(fft(x, fft_size) * np.conj(fft(y, fft_size)))
    cc = np.concatenate((cc[-(x_len-1):], cc[:x_len]))
    return np.real(cc) / den

def _ncc_c_3dim(x, y):

    """
    Variant of NCCc that operates with 2 dimensional X arrays and 2 dimensional
    y vector
    Returns a 3 dimensional array of normalized fourier transforms
    """
    den = norm(x, axis=1)[:, None] * norm(y, axis=1)
    den[den == 0] = np.Inf
    x_len = x.shape[-1]
    fft_size = 1 << (2*x_len-1).bit_length()
    cc = ifft(fft(x, fft_size) * np.conj(fft(y, fft_size))[:, None])
    cc = np.concatenate((cc[:,:,-(x_len-1):], cc[:,:,:x_len]), axis=2)
    return np.real(cc) / den.T[:, :, None]


def _sbd(x, y):
    # the same as python implementation
    ncc = _ncc_c(x, y)
    idx = ncc.argmax()
    dist = 1 - ncc[idx]
    yshift = roll_zeropad(y, (idx + 1) - max(len(x), len(y)))
    return dist, yshift


def _extract_shape(a,k):
    # the same as python implementation
    if len(a) == 0:
        return np.zeros((1, k))

        
    a = np.vstack(a)    
    columns = a.shape[1]
    y = zscore(a, axis=1, ddof=1)
    s = np.dot(y.transpose(), y)
    p = np.empty((columns, columns))
    p.fill(1.0/columns)
    p = np.eye(columns) - p
    m = np.dot(np.dot(p, s), p)
    _, vec = eigh(m)
    centroid = vec[:, -1]
    finddistance1 = math.sqrt(((a[0] - centroid) ** 2).sum())
    finddistance2 = math.sqrt(((a[0] + centroid) ** 2).sum())
    if finddistance1 >= finddistance2:
        centroid *= -1
    return zscore(centroid, ddof=1)

    

# helper function that is applied to each partition during extract_shape step of each iterations
# this function will first calc the closest centroid for each matrix and then consider this centroid as the current centroid in the extraction_shape step
#produce the grammatian matrix needed produced from timeseries in that partition
# then spark will aggregate these matrixes and calculate the new centroids    
def find_shape(centroids):
    def _find_shape(listOfSeries):
        numClusters = centroids.shape[0]
        feature_len = centroids.shape[1]
        
        cum_len = 0 
        temp = []
        for timeSeries in listOfSeries:
            cum_len+=1
            temp.append(timeSeries)
        gramMatrix = [np.zeros(feature_len+1) for _ in range(cum_len)]
        temp = np.vstack(temp)      
        distances = (1 - _ncc_c_3dim(temp, centroids).max(axis=2)).T
        idx = distances.argmin(1)
        for i in range(cum_len): 
            closest = idx[i]
            c = centroids[closest]
            if np.count_nonzero(c) == 0:
                series = temp[i]
            else: 
                _, series = _sbd(c,temp[i])            
            gramMatrix[i][0:-1] = series
            gramMatrix[i][-1] = idx[i]
                    
        return gramMatrix
    return _find_shape


# similar to the above function, except that it is used in the first time of shape extraction
# so it will use an initialized index of time series (currently i generate randomly, can later change it to the same one used in kmeans++)
def find_shape1(centroids):
    def _find_shape1(listOfSeries):
        numClusters = centroids.shape[0]
        feature_len = centroids.shape[1]
        idx = []
        temp = []
        cum_len = 0
        for timeSeries in listOfSeries:
            cum_len+=1
            temp.append(timeSeries[0:-1])
            idx.append(int(timeSeries[-1]))
        gramMatrix = [np.zeros(feature_len+1) for _ in range(cum_len)]   
        for i in range(cum_len):      
            (dist,xShift) = _sbd(centroids[idx[i]],temp[i])
            gramMatrix[i][0:-1] = xShift
            gramMatrix[i][-1] = idx[i]
        return gramMatrix
    return _find_shape1

#freqyebt directiosn
def fd2(ell,d,num_dict):
    def _fd(timeseries):
        res = [[] for i in range(num_dict)]
        m = 2 * ell
        temp = [[] for i in range(num_dict)]
        for x in timeseries:
            l = x.vector.toArray()
            idx = int(l[-1])
            temp[idx].append(l[0:-1])
        
        for idx,A in enumerate(temp):
            sketch = np.zeros((m, d))
            nextZeroRow = 0   
            for vector in A:
                if nextZeroRow >= m:
                    [_,s,Vt] = SVD(sketch , full_matrices=False)                   
                    if len(s) >= ell:
                        sShrunk = np.sqrt(s[:ell]**2 - s[ell-1]**2)
                        sketch[:ell:,:] = np.dot(np.diag(sShrunk), Vt[:ell,:])
                        sketch[ell:,:] = 0
                        nextZeroRow = ell
                    else:
                        sketch[:len(s),:] = np.dot(np.diag(s), Vt[:len(s),:])
                        sketch[len(s):,:] = 0
                        nextZeroRow = len(s)
                sketch[nextZeroRow,:] = vector 
                nextZeroRow += 1      
            [_,s,Vt] = SVD(sketch , full_matrices=False)
            if len(s) >= ell:
                    [_,s,Vt] = SVD(sketch , full_matrices=False)
                    sShrunk = np.sqrt(s[:ell]**2 - s[ell-1]**2)
                    sketch[:ell:,:] = np.dot(np.diag(sShrunk), Vt[:ell,:])
                    sketch[ell:,:] = 0      
            else:
                    [_,s,Vt] = SVD(sketch , full_matrices=False)
                    sketch[:len(s),:] = np.dot(np.diag(s), Vt[:len(s),:])
                    sketch[len(s):,:] = 0
            res[idx] = sketch[:ell,:]
        return zip(range(num_dict),res[idx])
    return _fd


def kshape(ell,inputTimeSeries,numPartitions,num_dict, maxIters=1):
    # explanation of input: k is length of each timesires, m is the number of timeseries,    
    k = inputTimeSeries.first().shape[0]

    
    m = inputTimeSeries.count()
    centroids = np.zeros((num_dict,k), dtype=float)
    sc = inputTimeSeries.context
    broadcast_numcluster = sc.broadcast(num_dict)
    broadcast_centroids = sc.broadcast(centroids)
    initialization = inputTimeSeries.map(lambda x: np.concatenate((x,[randint(0, broadcast_numcluster.value)])))   
    p = np.empty((k, k))
    p.fill(1.0/k)
    p = np.eye(k) - p
    p2 = np.zeros((k+1,k+1))
    p2[0:-1,0:-1] = p
    p2[-1][-1] = 1
    Q = DenseMatrix(k+1,k+1,p2.T.flatten()) 
    #initialization, compute first set of centroids based on randomly assigned index
    #returns each row as yshift against its centroid
    X = initialization.\
            mapPartitions(find_shape1(broadcast_centroids.value)).zipWithIndex()\
            .map(lambda x: (x[1],x[0])).cache()
    idx = X.map(lambda x: int(x[1][-1])).collect() 
    mat_X = IndexedRowMatrix(X)
    mat_X = mat_X.multiply(Q) 
    Y = mat_X.rows.cache()
    shapes = Y.mapPartitions(fd2(ell,k,num_dict)).reduceByKey(lambda a,b : a+b).\
            sortByKey().map(lambda x: x[1]).collect()
    
    new_centroids = np.vstack(shapes)    
    centroids = new_centroids    
    bcCentroids = sc.broadcast(centroids)
    
    for n in range(maxIters):
        
        start = time.time()
        #reverses the original step 
        #calculate index first and then use it to extract
        X = inputTimeSeries.\
            mapPartitions(find_shape(bcCentroids.value)).zipWithIndex().\
            map(lambda x: (x[1],x[0])).cache()
        mat_X = IndexedRowMatrix(X)
        mat_X = mat_X.multiply(Q)
        Y = mat_X.rows.cache()
        # use frequent direction to extract the new centroids 
        shapes = Y.mapPartitions(fd2(ell,k,num_dict)).reduceByKey(lambda a,b : a+b).\
                sortByKey().collect()
            
        new_centroids = np.vstack(shapes)          
        new_idx = X.map(lambda x: int(x[1][-1])).collect()
       
        if np.array_equal(new_idx, idx):
            print(n)
            break
        idx = new_idx
        if n == maxIters-1:
            break
        centroids = new_centroids
        end = time.time() 
        bcCentroids = sc.broadcast(centroids)
        #bcIndex = sc.broadcast(idx)
    cluster_idx = [[] for _ in range(num_dict)]
    print(new_idx)
    for i in range(m):
        cluster_idx[new_idx[i]].append(i+1)
    clusters = []
    
    for i, centroid in enumerate(centroids):
        clusters.append((centroid, cluster_idx[i]))

    return clusters




In [44]:
#the printed output is time elapsed 
#
spark = SparkSession\
        .builder\
        .master("spark://yiyangou-VirtualBox:7077")\
        .appName("kshape")\
        .config("spark.cores.max", "1")\
        .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

label = []
numPartitions = 6
#lines = sc.textFile("SITS1M_TRAIN.csv",16)
testData = sc.textFile("./Desktop/GRAIL4Spark/ECGFiveDays.csv",6).map(lambda x: np.array(list(map(float,x.split(",")))[0:-1])).cache()


#testData = lines.map(lambda x: np.array(list(map(float,x.split(",")))))
cluster_num=2
ell = 25
clusters = kshape(ell,testData,6, cluster_num)

dic = list(map(lambda x: x[0],clusters))
dic

136
(2, 136)
1231242
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 

[array([ 0.32953667, -0.00244101, -0.00244101, -0.00244101, -0.00244101,
        -0.00244101, -0.00244101, -0.00244101, -0.00244101, -0.00244101,
        -0.00244101, -0.00244101, -0.00244101, -0.00244101, -0.00244101,
        -0.00244101, -0.00244101, -0.00244101, -0.00244101, -0.00244101,
        -0.00244101, -0.00244101, -0.00244101, -0.00244101, -0.00244101,
        -0.00244101, -0.00244101, -0.00244101, -0.00244101, -0.00244101,
        -0.00244101, -0.00244101, -0.00244101, -0.00244101, -0.00244101,
        -0.00244101, -0.00244101, -0.00244101, -0.00244101, -0.00244101,
        -0.00244101, -0.00244101, -0.00244101, -0.00244101, -0.00244101,
        -0.00244101, -0.00244101, -0.00244101, -0.00244101, -0.00244101,
        -0.00244101, -0.00244101, -0.00244101, -0.00244101, -0.00244101,
        -0.00244101, -0.00244101, -0.00244101, -0.00244101, -0.00244101,
        -0.00244101, -0.00244101, -0.00244101, -0.00244101, -0.00244101,
        -0.00244101, -0.00244101, -0.00244101, -0.0

In [26]:
#compare kshape with kmeans
from sklearn.metrics.cluster import adjusted_rand_score
from pyspark.mllib.clustering import KMeans, KMeansModel
from sklearn.cluster import KMeans as sk_kmeans 


kshape_label = [0 for _ in range(884)]
print(len(kshape_label))
for i in clusters[0][1]:
    kshape_label[i] = 1
for i in clusters[1][1]:
    kshape_label[i] = 2
    
kmeans = sk_kmeans(n_clusters=2, random_state=0).fit(content)
kmeans_label= kmeans.predict(content)

print("kshape " +str(adjusted_rand_score(label,kshape_label)))

print("kmeans " +str(adjusted_rand_score(label,kmeans_label)))

884
kshape 0.7584429282730962
kmeans 0.00034732532830314124


In [54]:
def find_shape1(centroids):
    def _find_shape1(listOfSeries):
        numClusters = centroids.shape[0]
        feature_len = centroids.shape[1]
        idx = []
        temp = []
        cum_len = 0
        for timeSeries in listOfSeries:
            cum_len+=1
            temp.append(timeSeries[0:-1])
            idx.append(int(timeSeries[-1]))
        gramMatrix = [np.zeros(feature_len+1) for _ in range(cum_len)]   
        for i in range(cum_len):      
            (dist,xShift) = _sbd(centroids[idx[i]],temp[i])
            gramMatrix[i][0:-1] = xShift
            gramMatrix[i][-1] = idx[i]
        return gramMatrix
    return _find_shape1


inputTimeSeries = testData.map(lambda x: np.concatenate((x,[randint(0, cluster_num)])))\
                                          .cache()   
inputTimeSeries.count()
centroids = np.zeros((cluster_num,136), dtype=float)
X = inputTimeSeries.\
    mapPartitions(find_shape1(centroids)).count()
print(X)

884


In [5]:
randint(0, 100)

6